# Improved English to Khmer Transliteration with Advanced Techniques

This notebook implements several improvements over the baseline seq2seq model:
- **Attention Mechanism**: Helps decoder focus on relevant input parts
- **Bidirectional Encoder**: Captures context from both directions
- **Deeper Networks**: Multi-layer LSTMs for increased capacity
- **Beam Search**: Better quality decoding with beam width of 3
- **Data Augmentation**: Increases training data diversity
- **Cross-Validation**: K-fold validation for robust evaluation
- **Comprehensive Metrics**: BLEU, Character Error Rate (CER), Word Error Rate (WER)

In [3]:
import os
import numpy as np
import pandas as pd
import unicodedata
import re
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Attention, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Configuration parameters
EMBED_DIM = 64  # Increased from 32
LSTM_UNITS = 128  # Increased from 64
NUM_LAYERS = 2  # Multi-layer LSTMs
BATCH_SIZE = 32  # Increased batch size
EPOCHS = 100
BEAM_WIDTH = 3  # For beam search decoding
K_FOLDS = 5  # For cross-validation

# Paths
BASE_DIR = os.path.dirname(os.path.abspath('.'))
DATA_PATH = os.path.join(BASE_DIR, "data", "raw", "eng_khm_data.csv")
MODEL_PATH = os.path.join(BASE_DIR, "models", "english_romanizer.keras")
ASSETS_PATH = os.path.join(BASE_DIR, "data", "processed", "khmer_improved_assets.pkl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU Available: []


## Data Loading and Preprocessing with Augmentation

In [4]:
def augment_data(eng, khm):
    """Apply data augmentation techniques to increase diversity"""
    augmented = [(eng, khm)]  # Original pair
    
    # Augmentation 1: Add random noise (character substitution with low probability)
    if len(eng) > 2 and np.random.random() < 0.3:
        chars = list(eng)
        idx = np.random.randint(0, len(chars))
        # Small chance to duplicate a character
        if np.random.random() < 0.5:
            chars.insert(idx, chars[idx])
        augmented.append((''.join(chars), khm))
    
    return augmented

# Load and preprocess dataset
df = pd.read_csv(DATA_PATH)
print(f"Original dataset size: {len(df)}")

dataset = []
for _, row in df.iterrows():
    # Normalize English text
    normalized_eng = re.sub(r"[^a-z]", "", row['eng'].lower())
    
    # Normalize Khmer text
    normalized_khm = re.sub(r"[^\u1780-\u17FF]", "", row['khm'])
    normalized_khm = unicodedata.normalize('NFC', normalized_khm)
    
    if normalized_eng and normalized_khm:  # Only add non-empty pairs
        # Add original and augmented versions
        augmented_pairs = augment_data(normalized_eng, normalized_khm)
        dataset.extend(augmented_pairs)

print(f"Dataset size after augmentation: {len(dataset)}")
print(f"Sample pairs:")
for i in range(5):
    print(f"  {dataset[i][0]} -> {dataset[i][1]}")

Original dataset size: 28576
Dataset size after augmentation: 37024
Sample pairs:
  brodae -> ប្រដែ
  aasangkheyy -> អសង្ខៃយ
  chhatkophey -> ឆាតកភ័យ
  topvosompheareak -> ទព្វសម្ភារៈ
  topvosompheareak -> ទព្វសម្ភារៈ


In [5]:
# Tokenize English and Khmer texts
eng_tokenizer = Tokenizer(char_level=True, filters='', oov_token='<unk>')
eng_tokenizer.fit_on_texts([pair[0] for pair in dataset])

khm_tokenizer = Tokenizer(char_level=True, filters='', oov_token='<unk>')
khm_tokenizer.fit_on_texts(["\t", "\n"] + [pair[1] for pair in dataset])

print(f"English vocab size: {len(eng_tokenizer.word_index) + 1}")
print(f"Khmer vocab size: {len(khm_tokenizer.word_index) + 1}")

# Calculate max lengths
max_eng_len = max(len(pair[0]) for pair in dataset)
max_khm_len = max(len(pair[1]) for pair in dataset)
print(f"Max English length: {max_eng_len}")
print(f"Max Khmer length: {max_khm_len}")

English vocab size: 28
Khmer vocab size: 81
Max English length: 25
Max Khmer length: 24


In [6]:
# Create sequences for training
def prepare_sequences(dataset):
    encoder_inputs, decoder_inputs, decoder_targets = [], [], []
    
    for eng, khm in dataset:
        # Encoder sequence
        eng_seq = eng_tokenizer.texts_to_sequences([eng])[0]
        encoder_inputs.append(eng_seq)
        
        # Decoder sequences
        khm_seq = khm_tokenizer.texts_to_sequences([khm])[0]
        decoder_input = [khm_tokenizer.word_index['\t']] + khm_seq
        decoder_target = khm_seq + [khm_tokenizer.word_index['\n']]
        
        decoder_inputs.append(decoder_input)
        decoder_targets.append(decoder_target)
    
    encoder_data = pad_sequences(encoder_inputs, maxlen=max_eng_len, padding='post')
    decoder_input_data = pad_sequences(decoder_inputs, maxlen=max_khm_len + 1, padding='post')
    decoder_target_data = pad_sequences(decoder_targets, maxlen=max_khm_len + 1, padding='post')
    
    return encoder_data, decoder_input_data, decoder_target_data

encoder_data, decoder_input_data, decoder_target_data = prepare_sequences(dataset)
print(f"Encoder data shape: {encoder_data.shape}")
print(f"Decoder input shape: {decoder_input_data.shape}")
print(f"Decoder target shape: {decoder_target_data.shape}")

Encoder data shape: (37024, 25)
Decoder input shape: (37024, 25)
Decoder target shape: (37024, 25)


## Improved Model Architecture

Key improvements:
- **Bidirectional LSTM Encoder**: Captures context from both directions
- **Multi-layer LSTMs**: 2 layers for both encoder and decoder
- **Attention Mechanism**: Allows decoder to focus on relevant encoder outputs
- **Increased Capacity**: Larger embedding and hidden dimensions

In [7]:
def build_improved_model():
    """Build seq2seq model with attention, bidirectional encoder, and deeper networks"""
    
    # Encoder with Bidirectional LSTM
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    encoder_embedding = Embedding(
        input_dim=len(eng_tokenizer.word_index) + 1,
        output_dim=EMBED_DIM,
        mask_zero=True,
        name='encoder_embedding'
    )(encoder_inputs)
    
    # First bidirectional layer
    encoder_lstm1 = Bidirectional(
        LSTM(LSTM_UNITS, return_sequences=True, return_state=True),
        name='encoder_bilstm_1'
    )
    encoder_outputs1, forward_h1, forward_c1, backward_h1, backward_c1 = encoder_lstm1(encoder_embedding)
    
    # Second bidirectional layer
    encoder_lstm2 = Bidirectional(
        LSTM(LSTM_UNITS, return_sequences=True, return_state=True),
        name='encoder_bilstm_2'
    )
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm2(encoder_outputs1)
    
    # Combine forward and backward states
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])
    
    # Decoder with Attention
    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    decoder_embedding = Embedding(
        input_dim=len(khm_tokenizer.word_index) + 1,
        output_dim=EMBED_DIM,
        mask_zero=True,
        name='decoder_embedding'
    )(decoder_inputs)
    
    # First decoder LSTM layer
    decoder_lstm1 = LSTM(LSTM_UNITS * 2, return_sequences=True, return_state=True, name='decoder_lstm_1')
    decoder_outputs1, _, _ = decoder_lstm1(decoder_embedding, initial_state=[state_h, state_c])
    
    # Second decoder LSTM layer
    decoder_lstm2 = LSTM(LSTM_UNITS * 2, return_sequences=True, return_state=True, name='decoder_lstm_2')
    decoder_outputs, _, _ = decoder_lstm2(decoder_outputs1)
    
    # Attention layer
    attention = Attention(name='attention_layer')
    context_vector = attention([decoder_outputs, encoder_outputs])
    
    # Concatenate attention output with decoder output
    decoder_combined = Concatenate(name='concat_layer')([decoder_outputs, context_vector])
    
    # Output layer
    decoder_dense = Dense(
        len(khm_tokenizer.word_index) + 1,
        activation='softmax',
        name='decoder_dense'
    )
    decoder_outputs_final = decoder_dense(decoder_combined)
    
    # Build and compile model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs_final, name='improved_seq2seq')
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build the model
model = build_improved_model()
model.summary()

Model: "improved_seq2seq"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 64)  │      1,792 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_bilstm_1    │ [(None, None,     │    197,632 │ encoder_embeddin… │
│ (Bidirectional)     │ 256), (None,      │            │ not_equal[0][0]   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_bilstm_2    │ [(None, None,     │    394,240 │ encoder_bilstm_1… │
│ (Bidirectional)     │ 256), (None,      │            │ not_equal[0][0]   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 64)  │      5,184 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ encoder_bilstm_2… │
│ (Concatenate)       │                   │            │ encoder_bilstm_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ encoder_bilstm_2… │
│ (Concatenate)       │                   │            │ encoder_bilstm_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_1      │ [(None, None,     │    328,704 │ decoder_embeddin… │
│ (LSTM)              │ 256), (None,      │            │ concatenate[0][0… │
│                     │ 256), (None,      │            │ concatenate_1[0]… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None)      │          0 │ decoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_2      │ [(None, None,     │    525,312 │ decoder_lstm_1[0… │
│ (LSTM)              │ 256), (None,      │            │ not_equal_1[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_layer     │ (None, None, 256) │          0 │ decoder_lstm_2[0

 Total params: 1,494,417 (5.70 MB)

 Trainable params: 1,494,417 (5.70 MB)

 Non-trainable params: 0 (0.00 B)

## K-Fold Cross-Validation Training

Using 5-fold cross-validation for more robust performance estimates.

In [8]:
# Prepare for k-fold cross-validation
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
fold_histories = []
fold_scores = []

# Callbacks for training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print(f"Starting {K_FOLDS}-fold cross-validation...")
print(f"Total samples: {len(encoder_data)}")
print(f"Epochs per fold: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print("=" * 70)

Starting 5-fold cross-validation...
Total samples: 37024
Epochs per fold: 100
Batch size: 32


In [9]:
# K-fold cross-validation training loop
best_fold_idx = -1
best_val_loss = float('inf')

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(encoder_data)):
    print(f"\n{'=' * 70}")
    print(f"Training Fold {fold_idx + 1}/{K_FOLDS}")
    print(f"{'=' * 70}")
    
    # Split data
    encoder_train, encoder_val = encoder_data[train_idx], encoder_data[val_idx]
    decoder_input_train, decoder_input_val = decoder_input_data[train_idx], decoder_input_data[val_idx]
    decoder_target_train, decoder_target_val = decoder_target_data[train_idx], decoder_target_data[val_idx]
    
    print(f"Training samples: {len(encoder_train)}")
    print(f"Validation samples: {len(encoder_val)}")
    
    # Build fresh model for this fold
    fold_model = build_improved_model()
    
    # Train model
    history = fold_model.fit(
        [encoder_train, decoder_input_train],
        np.expand_dims(decoder_target_train, -1),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(
            [encoder_val, decoder_input_val],
            np.expand_dims(decoder_target_val, -1)
        ),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluate on validation set
    val_loss, val_accuracy = fold_model.evaluate(
        [encoder_val, decoder_input_val],
        np.expand_dims(decoder_target_val, -1),
        verbose=0
    )
    
    print(f"\nFold {fold_idx + 1} Results:")
    print(f"  Validation Loss: {val_loss:.4f}")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")
    
    fold_histories.append(history)
    fold_scores.append({'loss': val_loss, 'accuracy': val_accuracy})
    
    # Keep track of best fold
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_fold_idx = fold_idx
        # Save best model
        fold_model.save(MODEL_PATH)
        print(f"  ✓ Best model so far! Saved to {MODEL_PATH}")

print(f"\n{'=' * 70}")
print(f"Cross-Validation Complete!")
print(f"Best fold: {best_fold_idx + 1} with validation loss: {best_val_loss:.4f}")
print(f"{'=' * 70}")


Training Fold 1/5
Training samples: 29619
Validation samples: 7405
Epoch 1/100


2025-12-10 17:04:27.988915: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


459/926 ━━━━━━━━━━━━━━━━━━━━ 48s 104ms/step - accuracy: 0.2555 - loss: 3.1080

KeyboardInterrupt: 

In [ ]:
# Display cross-validation summary statistics
avg_val_loss = np.mean([score['loss'] for score in fold_scores])
std_val_loss = np.std([score['loss'] for score in fold_scores])
avg_val_accuracy = np.mean([score['accuracy'] for score in fold_scores])
std_val_accuracy = np.std([score['accuracy'] for score in fold_scores])

print("\nCross-Validation Summary:")
print(f"  Average Validation Loss: {avg_val_loss:.4f} ± {std_val_loss:.4f}")
print(f"  Average Validation Accuracy: {avg_val_accuracy:.4f} ± {std_val_accuracy:.4f}")
print("\nPer-Fold Results:")
for i, score in enumerate(fold_scores):
    print(f"  Fold {i+1}: Loss={score['loss']:.4f}, Accuracy={score['accuracy']:.4f}")

In [ ]:
# Save tokenizers and metadata
assets = {
    "eng_tokenizer": eng_tokenizer,
    "khm_tokenizer": khm_tokenizer,
    "max_eng_len": max_eng_len,
    "max_khm_len": max_khm_len,
    "fold_scores": fold_scores,
    "avg_val_loss": avg_val_loss,
    "avg_val_accuracy": avg_val_accuracy
}

with open(ASSETS_PATH, "wb") as file:
    pickle.dump(assets, file)

print(f"Assets saved to {ASSETS_PATH}")

## Inference with Beam Search Decoding

Implementing beam search for better quality predictions by exploring multiple candidate sequences.

In [ ]:
# Load the best model and assets
model = load_model(MODEL_PATH)
print(f"Loaded best model from {MODEL_PATH}")

with open(ASSETS_PATH, "rb") as file:
    assets = pickle.load(file)

eng_tokenizer = assets["eng_tokenizer"]
khm_tokenizer = assets["khm_tokenizer"]
max_eng_len = assets["max_eng_len"]
max_khm_len = assets["max_khm_len"]

print(f"Model loaded successfully!")
print(f"Average CV accuracy: {assets['avg_val_accuracy']:.4f}")

In [ ]:
# Build encoder model for inference
encoder_inputs = model.input[0]
encoder_outputs, forward_h1, forward_c1, backward_h1, backward_c1 = model.get_layer('encoder_bilstm_1').output

# Get outputs from second bidirectional layer
encoder_bilstm2_layer = model.get_layer('encoder_bilstm_2')
# We need to rebuild the encoder to get proper outputs
encoder_embedding_layer = model.get_layer('encoder_embedding')
encoder_bilstm1_layer = model.get_layer('encoder_bilstm_1')

# Rebuild encoder
x = encoder_embedding_layer(encoder_inputs)
x, fh1, fc1, bh1, bc1 = encoder_bilstm1_layer(x)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bilstm2_layer(x)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])
print("Encoder model built for inference")

In [ ]:
# Build decoder model for inference
decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(LSTM_UNITS * 2,))
decoder_state_input_c = Input(shape=(LSTM_UNITS * 2,))
encoder_outputs_input = Input(shape=(None, LSTM_UNITS * 2))

# Get decoder layers
decoder_embedding_layer = model.get_layer('decoder_embedding')
decoder_lstm1_layer = model.get_layer('decoder_lstm_1')
decoder_lstm2_layer = model.get_layer('decoder_lstm_2')
attention_layer = model.get_layer('attention_layer')
concat_layer = model.get_layer('concat_layer')
decoder_dense_layer = model.get_layer('decoder_dense')

# Build decoder inference path
decoder_embedded = decoder_embedding_layer(decoder_inputs)
decoder_out1, state_h1, state_c1 = decoder_lstm1_layer(
    decoder_embedded, 
    initial_state=[decoder_state_input_h, decoder_state_input_c]
)
decoder_out2, state_h2, state_c2 = decoder_lstm2_layer(decoder_out1)

# Attention
context = attention_layer([decoder_out2, encoder_outputs_input])
decoder_combined = concat_layer([decoder_out2, context])
decoder_outputs = decoder_dense_layer(decoder_combined)

decoder_model = Model(
    [decoder_inputs, encoder_outputs_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, state_h2, state_c2]
)
print("Decoder model built for inference")

In [ ]:
def beam_search_decode(input_text, beam_width=BEAM_WIDTH):
    """
    Perform beam search decoding for better quality outputs.
    
    Args:
        input_text: English text to transliterate
        beam_width: Number of beams to maintain
        
    Returns:
        Best decoded Khmer text
    """
    # Preprocess input
    text = str(input_text).strip()
    text = re.sub(r"[^a-z]", "", text.lower())
    
    if not text:
        return ""
    
    # Encode input
    seq = eng_tokenizer.texts_to_sequences([text])
    encoder_input = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    encoder_out, state_h, state_c = encoder_model.predict(encoder_input, verbose=0)
    
    # Start token
    start_token = khm_tokenizer.word_index['\t']
    end_token = khm_tokenizer.word_index['\n']
    
    # Initialize beams: (sequence, score, states)
    beams = [([start_token], 0.0, state_h, state_c)]
    
    for _ in range(max_khm_len + 1):
        all_candidates = []
        
        for seq, score, h, c in beams:
            # If sequence ended, keep it as is
            if seq[-1] == end_token:
                all_candidates.append((seq, score, h, c))
                continue
            
            # Get next token predictions
            target_seq = np.array([[seq[-1]]])
            predictions, new_h, new_c = decoder_model.predict(
                [target_seq, encoder_out, h, c], 
                verbose=0
            )
            
            # Get top k predictions
            top_k_indices = np.argsort(predictions[0, 0, :])[-beam_width:]
            
            for idx in top_k_indices:
                candidate_seq = seq + [idx]
                # Use log probability to avoid underflow
                candidate_score = score + np.log(predictions[0, 0, idx] + 1e-10)
                all_candidates.append((candidate_seq, candidate_score, new_h, new_c))
        
        # Select top beams
        ordered = sorted(all_candidates, key=lambda x: x[1], reverse=True)
        beams = ordered[:beam_width]
        
        # Stop if all beams ended
        if all(seq[-1] == end_token for seq, _, _, _ in beams):
            break
    
    # Return best sequence
    best_seq = beams[0][0]
    decoded_chars = []
    
    for idx in best_seq[1:]:  # Skip start token
        if idx == end_token:
            break
        char = khm_tokenizer.index_word.get(idx, '')
        if char:
            decoded_chars.append(char)
    
    return unicodedata.normalize('NFC', ''.join(decoded_chars))

print(f"Beam search decoder ready (beam width: {BEAM_WIDTH})")

## Comprehensive Evaluation Suite

Implementing standard metrics for transliteration quality:
- **BLEU Score**: Measures n-gram overlap with reference translations
- **Character Error Rate (CER)**: Edit distance at character level
- **Word Error Rate (WER)**: Accuracy at word level

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.metrics import edit_distance
import nltk

# Download necessary NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

def calculate_cer(reference, hypothesis):
    """Calculate Character Error Rate"""
    if len(reference) == 0:
        return 1.0 if len(hypothesis) > 0 else 0.0
    
    distance = edit_distance(reference, hypothesis)
    return distance / len(reference)

def calculate_wer(reference, hypothesis):
    """Calculate Word Error Rate"""
    ref_words = reference.split()
    hyp_words = hypothesis.split()
    
    if len(ref_words) == 0:
        return 1.0 if len(hyp_words) > 0 else 0.0
    
    distance = edit_distance(ref_words, hyp_words)
    return distance / len(ref_words)

def calculate_bleu(reference, hypothesis):
    """Calculate BLEU score with smoothing"""
    reference_tokens = list(reference)
    hypothesis_tokens = list(hypothesis)
    
    if len(hypothesis_tokens) == 0:
        return 0.0
    
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing)

def evaluate_model(test_pairs, beam_width=BEAM_WIDTH, max_samples=None):
    """
    Evaluate model on test pairs with comprehensive metrics.
    
    Args:
        test_pairs: List of (english, khmer) tuples
        beam_width: Beam width for decoding
        max_samples: Maximum number of samples to evaluate (None for all)
        
    Returns:
        Dictionary with evaluation metrics
    """
    if max_samples:
        test_pairs = test_pairs[:max_samples]
    
    bleu_scores = []
    cer_scores = []
    wer_scores = []
    exact_matches = 0
    
    print(f"Evaluating on {len(test_pairs)} samples with beam width {beam_width}...")
    
    for i, (eng, khm_ref) in enumerate(test_pairs):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i + 1}/{len(test_pairs)} samples...")
        
        # Generate prediction
        khm_pred = beam_search_decode(eng, beam_width=beam_width)
        
        # Calculate metrics
        bleu = calculate_bleu(khm_ref, khm_pred)
        cer = calculate_cer(khm_ref, khm_pred)
        wer = calculate_wer(khm_ref, khm_pred)
        
        bleu_scores.append(bleu)
        cer_scores.append(cer)
        wer_scores.append(wer)
        
        if khm_pred == khm_ref:
            exact_matches += 1
    
    results = {
        'bleu': {
            'mean': np.mean(bleu_scores),
            'std': np.std(bleu_scores),
            'median': np.median(bleu_scores)
        },
        'cer': {
            'mean': np.mean(cer_scores),
            'std': np.std(cer_scores),
            'median': np.median(cer_scores)
        },
        'wer': {
            'mean': np.mean(wer_scores),
            'std': np.std(wer_scores),
            'median': np.median(wer_scores)
        },
        'exact_match_rate': exact_matches / len(test_pairs),
        'num_samples': len(test_pairs)
    }
    
    return results

print("Evaluation functions defined successfully!")

In [ ]:
# Create test set from the original dataset (use a held-out portion)
np.random.seed(42)
test_size = min(500, len(dataset) // 5)  # 20% or max 500 samples
test_indices = np.random.choice(len(dataset), size=test_size, replace=False)
test_pairs = [dataset[i] for i in test_indices]

print(f"Test set size: {len(test_pairs)} samples")
print("\nSample test pairs:")
for i in range(5):
    print(f"  {test_pairs[i][0]} -> {test_pairs[i][1]}")

In [ ]:
# Run comprehensive evaluation
eval_results = evaluate_model(test_pairs, beam_width=BEAM_WIDTH)

print("\n" + "=" * 70)
print("EVALUATION RESULTS")
print("=" * 70)
print(f"\nTest Set Size: {eval_results['num_samples']} samples")
print(f"Beam Width: {BEAM_WIDTH}")
print("\nMetrics:")
print(f"  BLEU Score:")
print(f"    Mean:   {eval_results['bleu']['mean']:.4f} ± {eval_results['bleu']['std']:.4f}")
print(f"    Median: {eval_results['bleu']['median']:.4f}")
print(f"\n  Character Error Rate (CER):")
print(f"    Mean:   {eval_results['cer']['mean']:.4f} ± {eval_results['cer']['std']:.4f}")
print(f"    Median: {eval_results['cer']['median']:.4f}")
print(f"\n  Word Error Rate (WER):")
print(f"    Mean:   {eval_results['wer']['mean']:.4f} ± {eval_results['wer']['std']:.4f}")
print(f"    Median: {eval_results['wer']['median']:.4f}")
print(f"\n  Exact Match Rate: {eval_results['exact_match_rate']:.2%}")
print("=" * 70)

## Visualization of Results

In [ ]:
# Plot training history for best fold
best_history = fold_histories[best_fold_idx]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss plot
axes[0, 0].plot(best_history.history['loss'], label='Training Loss', linewidth=2)
axes[0, 0].plot(best_history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0, 0].set_title(f'Training History - Best Fold ({best_fold_idx + 1})', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy plot
axes[0, 1].plot(best_history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0, 1].plot(best_history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0, 1].set_title('Accuracy Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Cross-validation scores
fold_numbers = list(range(1, K_FOLDS + 1))
fold_losses = [score['loss'] for score in fold_scores]
fold_accs = [score['accuracy'] for score in fold_scores]

axes[1, 0].bar(fold_numbers, fold_losses, color='steelblue', alpha=0.7)
axes[1, 0].axhline(y=avg_val_loss, color='red', linestyle='--', label=f'Mean: {avg_val_loss:.4f}')
axes[1, 0].set_title('Validation Loss per Fold', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Fold')
axes[1, 0].set_ylabel('Validation Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

axes[1, 1].bar(fold_numbers, fold_accs, color='forestgreen', alpha=0.7)
axes[1, 1].axhline(y=avg_val_accuracy, color='red', linestyle='--', label=f'Mean: {avg_val_accuracy:.4f}')
axes[1, 1].set_title('Validation Accuracy per Fold', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Fold')
axes[1, 1].set_ylabel('Validation Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize evaluation metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['BLEU', 'CER', 'WER']
metric_keys = ['bleu', 'cer', 'wer']
colors = ['steelblue', 'coral', 'mediumseagreen']

for idx, (metric, key, color) in enumerate(zip(metrics, metric_keys, colors)):
    mean_val = eval_results[key]['mean']
    std_val = eval_results[key]['std']
    median_val = eval_results[key]['median']
    
    axes[idx].bar(['Mean', 'Median'], [mean_val, median_val], color=color, alpha=0.7, width=0.5)
    axes[idx].errorbar(['Mean'], [mean_val], yerr=[std_val], fmt='none', color='black', capsize=10, capthick=2)
    axes[idx].set_title(f'{metric} Score', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('Score')
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for i, v in enumerate([mean_val, median_val]):
        axes[idx].text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nExact Match Rate: {eval_results['exact_match_rate']:.2%}")

## Example Predictions

Let's test the improved model with beam search on some examples.

In [ ]:
# Test examples
test_examples = [
    "hello",
    "trap",
    "mean luy",
    "kdas",
    "cambodia",
    "phnom penh",
    "angkor wat",
    "khmer"
]

print("=" * 70)
print("EXAMPLE PREDICTIONS WITH BEAM SEARCH")
print("=" * 70)

for example in test_examples:
    result = beam_search_decode(example, beam_width=BEAM_WIDTH)
    print(f"\nInput:  {example}")
    print(f"Output: {result}")

In [ ]:
# Show some example predictions with references from test set
print("\n" + "=" * 70)
print("COMPARISON WITH REFERENCE TRANSLATIONS")
print("=" * 70)

np.random.seed(123)
sample_indices = np.random.choice(len(test_pairs), size=min(10, len(test_pairs)), replace=False)

for idx in sample_indices:
    eng, khm_ref = test_pairs[idx]
    khm_pred = beam_search_decode(eng, beam_width=BEAM_WIDTH)
    
    # Calculate metrics for this example
    bleu = calculate_bleu(khm_ref, khm_pred)
    cer = calculate_cer(khm_ref, khm_pred)
    
    print(f"\nInput:      {eng}")
    print(f"Reference:  {khm_ref}")
    print(f"Prediction: {khm_pred}")
    print(f"BLEU: {bleu:.4f} | CER: {cer:.4f} | Match: {'✓' if khm_ref == khm_pred else '✗'}")

## Summary of Improvements

This improved model implements:

1. **Attention Mechanism**: The attention layer allows the decoder to dynamically focus on relevant parts of the input sequence, significantly improving translation quality for longer sequences.

2. **Bidirectional Encoder**: Using bidirectional LSTMs in the encoder captures context from both past and future, providing richer representations.

3. **Deeper Networks**: 2-layer LSTMs in both encoder and decoder increase model capacity to learn complex patterns.

4. **Beam Search Decoding**: Exploring top-3 candidate sequences during inference produces higher quality outputs compared to greedy decoding.

5. **Data Augmentation**: Character-level augmentation increases training data diversity and model robustness.

6. **K-Fold Cross-Validation**: 5-fold cross-validation provides more reliable performance estimates and helps prevent overfitting.

7. **Comprehensive Evaluation**: Standard metrics (BLEU, CER, WER) allow proper comparison with other transliteration systems.

8. **Increased Model Capacity**: Larger embedding dimensions (64 vs 32) and LSTM units (128 vs 64) allow the model to learn more complex mappings.

### Key Benefits:
- More robust training with cross-validation
- Better sequence modeling with attention and bidirectional processing
- Higher quality predictions with beam search
- Comprehensive evaluation for proper performance assessment
- Increased model capacity for complex patterns